# Aanbod middenhuurwoningen in Nederland

In [60]:
import pandas as pd

pararius = pd.read_csv("../data/pararius_listings.csv")
pararius.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2112 entries, 0 to 2111
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Link              2112 non-null   object
 1   Huurprijs         2110 non-null   object
 2   Locatie           2110 non-null   object
 3   m2                2112 non-null   object
 4   Huurovereenkomst  1064 non-null   object
 5   Beschrijving      2110 non-null   object
dtypes: object(6)
memory usage: 99.1+ KB


### Opschonen

Om de dataset te kunnen analyseren moeten eerst een aantal kolommen een ander datatype krijgen, worden opgeschoond en waar nodig worden bijgevuld:

1. Er zijn twee links die geen waarden hebben voor `Huurprijs`, `Locatie` en `Beschrijving`. Deze links werken niet meer dus we verwijderen deze rijen.
2. De `Huurprijs` kolom moet worden omgezet naar een `float`.
    - Sommige van deze waarden worden als volgt weergeven: '€ 1.075 - 1.775 per month'. In dat geval berekenen we het gemiddelde van de prijs.
    - Voor sommige huurwoningen moet je de prijs opvragen, de waarde is dan 'Price on request'. Voor dat geval voegen we een nieuwe kolom toe `Verzoek` met als waarde `True` of `False`.
3. De `m2` moet ook worden omgezet naar `float`.
    - Hier geldt dezelfde uitzondering als bij de huurprijs. We berekenen ook het gemiddelde aantal vierkante meter in dit geval.
4. De `Locatie` kolom bevat een postcode en de naam van de buurt. Dit gaan we opsplitsen in twee kolommen.

In [61]:
# Vind de lege rijen en verwijder ze
lege_rijen = pararius[pararius[['Huurprijs', 'Locatie', 'Beschrijving']].isnull().all(axis=1)]
pararius = pararius.drop(lege_rijen.index)

In [62]:
def schoon_huurprijs(prijs):
    """Functie om de 'Huurprijs' kolom op te schonen."""
    if isinstance(prijs, float) or prijs is None:
        return prijs
    
    if "Price on request" in str(prijs):
        return None # Lege waarde voor de huurprijs
    
    prijs = prijs.replace("€", "").replace("per month", "").replace(",", "").strip()

    # Check of de huurprijs twee prijzen kan zijn
    if "-" in prijs:
        laag, hoog = prijs.split("-")
        laag = float(laag.strip())
        hoog = float(hoog.strip())
        return (laag + hoog) / 2 # Bereken het gemiddelde van de twee prijzen
    else:
        return float(prijs)


def schoon_m2(waarde):
    """Functie om de 'm2' kolom op te schonen."""
    if pd.isnull(waarde):
        return None
    
    waarde = waarde.replace("m²", "").replace("㎡", "").strip()

    # Bereken het gemiddelde als er twee waarden voor 'm2' zijn
    if "-" in waarde:
        laag, hoog = waarde.split("-")
        laag = laag.strip()
        hoog = hoog.strip()
        return (float(laag) + float(hoog)) / 2
    else:
        return float(waarde)


def krijg_buurt(locatie):
    """Functie die de 'Locatie' kolom opsplits in de postcode en buurt."""
    if pd.isnull(locatie):
        return None
    
    if "(" in locatie and ")" in locatie:
        buurt = locatie.split("(")[-1].split(")")[0].strip()
        return buurt
    else:
        return None


# Huurprijs opschonen
pararius["Verzoek"] = pararius["Huurprijs"].apply(lambda x: True if "Price on request" in str(x) else False)
pararius["Huurprijs"] = pararius["Huurprijs"].apply(schoon_huurprijs)

# Vierkante meter opschonen
pararius["m2"] = pararius["m2"].apply(schoon_m2)

# Locatie opschonen
pararius['Buurt'] = pararius['Locatie'].apply(krijg_buurt)
pararius['Locatie'] = pararius['Locatie'].apply(lambda x: x.split('(')[0].strip() if pd.notnull(x) else x)

pararius.head()